# LABORATORIO N°8
### Gutierrez Coronado Matias Ezequiel
### Ingeniería de Sistemas

# Entorno brazo robótico

Este entorno simula un brazo KUKA iiwa en PyBullet, cuyo objetivo principal es voltear un vaso que aparece en posiciones aleatorias sobre una mesa. Se proporcionan observaciones físicas relevantes y se recompensa únicamente cuando el vaso es volcado, mientras que las acciones ineficientes o los contactos indebidos se penalizan. Es una plataforma ideal para probar y entrenar algoritmos de aprendizaje por refuerzo aplicados al control robótico.

In [ ]:
import pybullet as p
import pybullet_data
import numpy as np
import gym
from gym import spaces

class RoboticCupEnv(gym.Env):
    def __init__(self, render_mode='human'):
        super().__init__()
        if render_mode == 'human':
            self.physicsClient = p.connect(p.GUI)
        else:
            self.physicsClient = p.connect(p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)
        self.planeId = p.loadURDF("plane.urdf")
        self.robot_id = p.loadURDF("kuka_iiwa/model.urdf", useFixedBase=True)
        self.cup_radius = 0.035
        self.cup_height = 0.13
        self.cup_id = None

        self.x_range = (0.2, 0.9)
        self.init_robot_pose = [0, 0, 0, 0, 0, 0, 0]
        self.base_action_range = 0.05
        self.max_action_range = 0.25  # Permitimos aumentar más en etapa final
        self.action_range = self.base_action_range
        self.action_space = spaces.Box(
            low=-self.action_range, high=self.action_range, shape=(7,), dtype=np.float32
        )
        self.observation_space = spaces.Box(
            low=np.array([self.x_range[0], self.x_range[0], 0] + [-3.14]*7),
            high=np.array([self.x_range[1], self.x_range[1], 1.5] + [3.14]*7),
            dtype=np.float32
        )
        self.max_steps = 900
        self.last_ee_pos = None
        self.stagnation_counter = 0
        self.visited_x = set()
        self.reset()

    def reset(self):
        p.resetSimulation()
        p.setGravity(0, 0, -9.81)
        self.planeId = p.loadURDF("plane.urdf")
        self.robot_id = p.loadURDF("kuka_iiwa/model.urdf", useFixedBase=True)
        x_cup = np.random.uniform(*self.x_range)
        y_cup = 0
        z_cup = self.cup_height / 2
        cup_col = p.createCollisionShape(p.GEOM_CYLINDER, radius=self.cup_radius, height=self.cup_height)
        cup_vis = p.createVisualShape(p.GEOM_CYLINDER, radius=self.cup_radius, length=self.cup_height, rgbaColor=[1,1,1,1])
        self.cup_id = p.createMultiBody(
            baseMass=0.2,
            baseCollisionShapeIndex=cup_col,
            baseVisualShapeIndex=cup_vis,
            basePosition=[x_cup, y_cup, z_cup]
        )
        for i in range(7):
            p.resetJointState(self.robot_id, i, self.init_robot_pose[i])
        self.steps = 0
        self.action_range = self.base_action_range
        self.action_space = spaces.Box(
            low=-self.action_range, high=self.action_range, shape=(7,), dtype=np.float32
        )
        self.last_ee_pos = None
        self.stagnation_counter = 0
        self.visited_x = set()

        # Ajuste automático de cámara
        if p.getConnectionInfo()['connectionMethod'] == p.GUI:
            p.resetDebugVisualizerCamera(
                cameraDistance=1.2,
                cameraYaw=90,
                cameraPitch=-30,
                cameraTargetPosition=[0.55, 0, 0.25]
            )
        return self._get_obs()

    def _get_obs(self):
        cup_pos, _ = p.getBasePositionAndOrientation(self.cup_id)
        ee_pos = p.getLinkState(self.robot_id, 6)[0]
        dist = abs(ee_pos[0] - cup_pos[0])
        joint_states = [p.getJointState(self.robot_id, i)[0] for i in range(7)]
        return np.array([cup_pos[0], ee_pos[0], dist] + list(joint_states), dtype=np.float32)

    def step(self, action):
        # 1. Aumenta velocidad de acción tras 40% y 70% si lejos
        obs = self._get_obs()
        dist = obs[2]
        if self.steps > int(self.max_steps * 0.4) and dist > 0.15:
            self.action_range = min(self.max_action_range, self.action_range + 0.01)
        if self.steps > int(self.max_steps * 0.7) and dist > 0.18:
            self.action_range = min(self.max_action_range, self.action_range + 0.02)
        self.action_space = spaces.Box(
            low=-self.action_range, high=self.action_range, shape=(7,), dtype=np.float32
        )

        # Aplica acción
        for i in range(7):
            current_angle = p.getJointState(self.robot_id, i)[0]
            p.setJointMotorControl2(
                self.robot_id, i, p.POSITION_CONTROL,
                targetPosition=current_angle + action[i], force=100
            )
        for _ in range(5):
            p.stepSimulation()
        self.steps += 1

        obs = self._get_obs()
        dist = obs[2]

        # Early stop: al 80% pasos, si sigue muy lejos termina episodio y penaliza fuerte
        done = False
        early_stop = False
        if self.steps > int(self.max_steps * 0.8) and dist > 0.22:
            done = True
            early_stop = True

        reward, done_step, info = self._compute_reward_done(obs, action, early_stop)
        done = done or done_step

        # ---------- Penalización por volver a un lugar lejos del vaso ----------
        mano_x = round(obs[1], 2)  # obs[1] es ee_x
        if (mano_x in self.visited_x) and (dist > 0.04):  # solo penaliza lejos (>4cm)
            reward -= 5
        self.visited_x.add(mano_x)
        # ----------------------------------------------------------------------

        return obs, reward, done, info

    def _compute_reward_done(self, obs, action, early_stop=False):
        cup_x, ee_x, dist = obs[0], obs[1], obs[2]
        cup_ori = p.getBasePositionAndOrientation(self.cup_id)[1]
        cup_euler = p.getEulerFromQuaternion(cup_ori)
        flipped = (abs(cup_euler[0]) > (np.pi/2)) or (abs(cup_euler[1]) > (np.pi/2))

        # Penalización base por paso (más fuerte tras el 60%)
        reward = -0.07 if self.steps > int(self.max_steps * 0.6) else -0.05

        # Recompensa densa, menor tras el 60% de pasos
        if self.steps > int(self.max_steps * 0.6):
            reward += 0.01 / (1.0 + dist)
        else:
            reward += 0.04 / (1.0 + dist)

        # Bonus pequeño por estar muy cerca del vaso
        if dist < 0.05:
            reward += 0.8

        cup_pos, _ = p.getBasePositionAndOrientation(self.cup_id)

        # Penaliza contacto de cualquier parte (excepto la mano) con el vaso
        for i in range(6):  # links 0 a 5
            link_pos = p.getLinkState(self.robot_id, i)[0]
            link_dist = np.linalg.norm(np.array(link_pos) - np.array(cup_pos))
            if link_dist < 0.04:
                reward -= 50 if self.steps > int(self.max_steps * 0.6) else 30

        # Penaliza si cualquier link (excepto la mano) toca el suelo
        for i in range(6):
            link_pos = p.getLinkState(self.robot_id, i)[0]
            if link_pos[2] < 0.05:
                reward -= 30 if self.steps > int(self.max_steps * 0.6) else 20

        # Penaliza si la mano está muy alta y no ha volcado el vaso
        mano_pos = p.getLinkState(self.robot_id, 6)[0]
        mano_dist = np.linalg.norm(np.array(mano_pos) - np.array(cup_pos))
        if mano_pos[2] > 0.08 and not flipped:
            reward -= 3

        # Bonus por contacto de la mano con el vaso (no acumulativo)
        contacto_con_mano = mano_dist < 0.04
        if contacto_con_mano:
            reward += 5

        # Penaliza si se queda estancado (sin moverse suficientemente)
        if self.last_ee_pos is not None:
            if np.linalg.norm(np.array(mano_pos) - np.array(self.last_ee_pos)) < 0.01:  # más exigente
                self.stagnation_counter += 1
                if self.stagnation_counter > 20:  # menos pasos permitidos en "idle"
                    reward -= 30
            else:
                self.stagnation_counter = 0
        self.last_ee_pos = mano_pos

        # === SOLO IMPORTA QUE SE VOLTEE EL VASO ===
        if flipped:
            reward += 1000 + (self.max_steps - self.steps)
            return reward, True, {'success': True}
        # Early stop
        if early_stop:
            reward -= 400
            return reward, True, {'success': False, 'early_stop': True}
        # Penalización muy dura si no logra la tarea
        if self.steps >= self.max_steps:
            reward -= 250
            return reward, True, {'success': False}
        return reward, False, {}

    def render(self, mode='human'):
        pass

    def close(self):
        p.disconnect()

## Recompensas
El entorno recompensa al agente por acercarse al vaso y otorga un gran premio si logra volcarlo. También suma puntos si la mano toca el vaso.
Por otro lado, penaliza cada paso, los toques con otras partes del brazo, si el brazo toca el suelo, si la mano está muy alta sin éxito, por moverse poco, y por volver a posiciones alejadas del vaso.
Si no logra la tarea dentro del límite de pasos, la penalización es aún mayor.
Esto motiva al agente a actuar rápido, con precisión y sin movimientos innecesarios.

# Entrenamiento PPO con TensorBoard

Este bloque configura y entrena un modelo PPO sobre el entorno `RoboticCupEnv`, utilizando una arquitectura de red personalizada y registrando las métricas de entrenamiento en TensorBoard para su análisis visual. Así es posible monitorear el proceso de aprendizaje en tiempo real y ajustar los hiperparámetros de manera informada.


In [ ]:
from stable_baselines3 import PPO

policy_kwargs = dict(net_arch=[256, 128])
env = RoboticCupEnv(render_mode='DIRECT')

tensorboard_log_dir = "./tensorboard_logs/brazorobotico/"

model = PPO(
    'MlpPolicy',
    env,
    verbose=1,
    ent_coef=0.02,
    policy_kwargs=policy_kwargs,
    n_steps=4096,
    batch_size=256,
    learning_rate=3e-4,
    tensorboard_log=tensorboard_log_dir 
)

model.learn(total_timesteps=1000000)
model.save("ppo_brazo_robotico_vf")
env.close()





c:\Users\Legion 5 Pro\OneDrive\Documents\SIS420\myenv\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
c:\Users\Legion 5 Pro\OneDrive\Documents\SIS420\myenv\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./tensorboard_logs/brazorobotico/PPO_4


c:\Users\Legion 5 Pro\OneDrive\Documents\SIS420\myenv\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


----------------------------------
| rollout/           |           |
|    ep_len_mean     | 650       |
|    ep_rew_mean     | -4.03e+03 |
| time/              |           |
|    fps             | 356       |
|    iterations      | 1         |
|    time_elapsed    | 11        |
|    total_timesteps | 4096      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 600          |
|    ep_rew_mean          | -3.26e+03    |
| time/                   |              |
|    fps                  | 330          |
|    iterations           | 2            |
|    time_elapsed         | 24           |
|    total_timesteps      | 8192         |
| train/                  |              |
|    approx_kl            | 0.0073125865 |
|    clip_fraction        | 0.0766       |
|    clip_range           | 0.2          |
|    entropy_loss         | -9.96        |
|    explained_variance   | 0.00125      |
|    

# Evaluación de la política entrenada

Este script carga el modelo PPO previamente entrenado y ejecuta varias simulaciones en el entorno físico para verificar la capacidad del agente de voltear el vaso. Al final, se imprime un resumen de los éxitos obtenidos durante las pruebas.


In [6]:
import time
from stable_baselines3 import PPO

env = RoboticCupEnv(render_mode='human')
model = PPO.load("ppo_brazo_robotico_vf", env=env)

num_episodios = 5
resultados = []

for ep in range(num_episodios):
    obs = env.reset()
    done = False
    info = {}
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        time.sleep(0.05)  # Para visualizar el movimiento

    print(f"\nEpisodio {ep+1} - Info final: {info}")
    exito = info.get('success', False)
    print(f"¿Éxito? {'✅' if exito else '❌'}")
    resultados.append(exito)

env.close()
print("\nResumen de éxitos en 5 episodios:", resultados)
print(f"Total de éxitos: {sum(resultados)} / {num_episodios}")



c:\Users\Legion 5 Pro\OneDrive\Documents\SIS420\myenv\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
c:\Users\Legion 5 Pro\OneDrive\Documents\SIS420\myenv\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
c:\Users\Legion 5 Pro\OneDrive\Documents\SIS420\myenv\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Episodio 1 - Info final: {'success': True}
¿Éxito? ✅

Episodio 2 - Info final: {'success': True}
¿Éxito? ✅

Episodio 3 - Info final: {'success': True}
¿Éxito? ✅

Episodio 4 - Info final: {'success': True}
¿Éxito? ✅

Episodio 5 - Info final: {'success': True}
¿Éxito? ✅

Resumen de éxitos en 5 episodios: [True, True, True, True, True]
Total de éxitos: 5 / 5


# Visualización de métricas con TensorBoard

Durante el entrenamiento, el modelo PPO guarda de forma automática varias métricas relevantes en un archivo de logs compatible con TensorBoard. Entre estas métricas se incluyen la recompensa promedio por episodio, la entropía de la política, el explained variance y otros indicadores clave del aprendizaje. Estos datos permiten visualizar en tiempo real el progreso y la estabilidad del entrenamiento, ayudando a identificar mejoras o problemas en el proceso de aprendizaje del agente. Para analizar estos resultados basta con iniciar TensorBoard apuntando al directorio donde se guardaron los logs.
Se obtienen los datos colocando por consola el siguiente comando: `tensorboard --logdir=./tensorboard_logs/brazorobotico/`

# Evolución de la longitud de episodios

La siguiente gráfica representa cómo varió la cantidad promedio de pasos necesarios para que el brazo robótico complete la tarea de voltear el vaso a lo largo del entrenamiento. Al principio, el agente necesita muchos pasos por episodio porque no sabe cómo resolver el reto. A medida que aprende y mejora su estrategia, la longitud promedio de los episodios disminuye de manera constante, lo que indica que el brazo es capaz de cumplir el objetivo de forma más eficiente y en menos tiempo. Esta tendencia descendente valida que el aprendizaje por refuerzo está funcionando correctamente en el entorno diseñado.

![Longitud promedio de episodios](graficas/log_episodios.png)


# Evolución de la recompensa promedio

Esta gráfica muestra la evolución de la recompensa promedio por episodio obtenida por el brazo robótico durante el entrenamiento. Al inicio, el valor de la recompensa es muy bajo debido a las múltiples penalizaciones y la dificultad del agente para resolver la tarea. Sin embargo, a medida que el aprendizaje progresa, la línea asciende consistentemente, acercándose e incluso superando valores positivos. Esto significa que el agente está mejorando su comportamiento: comete menos errores y logra tumbar el vaso de forma más eficiente, optimizando las acciones para maximizar la recompensa total de cada episodio.

![Recompensa promedio de episodios](graficas/rec_episodios.png)


## Gráfica de explained_variance

Esta gráfica muestra cómo la métrica **explained_variance** evoluciona durante el entrenamiento del agente PPO para el brazo robótico. Un valor cercano a 1 indica que el modelo puede predecir de forma precisa las recompensas futuras, mientras que valores bajos reflejan incertidumbre en las predicciones. En este experimento, se observa que el explained_variance aumenta y se estabiliza cerca de 0.9, lo que demuestra que la red neuronal está aprendiendo correctamente a estimar el valor de los estados alcanzados durante la tarea.
![Explained Variance](graficas/exp_var.png)

## Evolución del policy_gradient_loss

Esta gráfica muestra cómo evoluciona el `policy_gradient_loss` durante el entrenamiento del brazo robótico. Al principio el valor es más negativo y variable, reflejando grandes ajustes en la política del agente mientras explora y aprende a resolver la tarea. A medida que el entrenamiento avanza, el valor tiende a estabilizarse cerca de cero, lo que indica que la política del agente ya está optimizada y requiere menos correcciones para lograr su objetivo de voltear el vaso de manera eficiente.

![Evolución del policy_gradient_loss](graficas/pol_gradiente.png)



## Conclusión

A lo largo del entrenamiento, el agente de control para el brazo robótico demostró una mejora significativa en su desempeño al optimizar la política de acciones. Las métricas analizadas reflejan un proceso de aprendizaje estable y eficiente, logrando que el agente cumpla el objetivo de voltear el vaso de manera más precisa y consistente.
